Notebook for performance analysis with PostGIS
=============================================

Prerequisite:
* python and ipython notebook installed
* psycopg2 (although this is optional as loading can also be done straight in SQL)
* extension *sql* magic (see https://github.com/catherinedevlin/ipython-sql for details)


Also, one needs a postgres/postgis instance running and adjust the connection string.

In [1]:
%load_ext sql

In [2]:
%sql postgresql://localhost/pspat

u'Connected: None@pspat'

Data loading strategy for brain region and neuron (3D box)
==========================================================

* Data access object layer (DAO java-code) provides a method to convert input coordinate of opposite corners (lower-left, upper-right) into the WKT string (see an ex. 'objAsWKT' below)
* The DAO insert method load Polyhedral cube using ST_GeomFromEWKT(%cube) by passing the string conversion (see insertSQL)
* possible methods overloaded possible to take alternative input: extreme corner, centroid and size


**DDL test set-up** (use a simple 'cube' table for test purposes)

In [4]:
%%sql
drop table cube;
create table cube(id serial
                   ,name varchar(100)
                   ,centroid_x double precision
                   ,centroid_y double precision
                   ,centroid_z double precision);

Done.
Done.


[]

In [5]:
%%sql 
select AddGeometryColumn('public','cube','obj_geom',0,'POLYHEDRALSURFACE',3);


1 rows affected.


addgeometrycolumn
public.cube.obj_geom SRID:0 TYPE:POLYHEDRALSURFACE DIMS:3


In [6]:
%%sql 
alter table cube alter column obj_geom set not null;
create index cubeindex on cube using gist(obj_geom);
create unique index on cube(name);

Done.
Done.
Done.


[]

In [7]:
def objAsWKT(minCoord, maxCoord):
    """ minCoord: tupple for min coordinates of 3D box
        maxCoord: tupple for max coordinates of 3D box
    """
    xmin = str(minCoord[0]) + " "
    ymin = str(minCoord[1]) + " "
    zmin = str(minCoord[2]) + " "
    xmax = str(maxCoord[0]) + " "
    ymax = str(maxCoord[1]) + " " 
    zmax = str(maxCoord[2]) + " "
    
    prefix = 'SRID=0; POLYHEDRALSURFACE('
    #facets vertices counter-clockwise
    z = '((' +xmin+ymin+zmax+','+xmax+ymin+zmax+','+xmax+ymax+zmax+','+xmin+ymax+zmax+','+xmin+ymin+zmax + ')),'
    z_= '((' +xmax+ymin+zmin+','+xmin+ymin+zmin+','+xmin+ymax+zmin+','+xmax+ymax+zmin+','+xmax+ymin+zmin + ')),'
    x = '((' +xmax+ymin+zmax+','+xmax+ymin+zmin+','+xmax+ymax+zmin+','+xmax+ymax+zmax+','+xmax+ymin+zmax + ')),'
    x_= '((' +xmin+ymin+zmin+','+xmin+ymin+zmax+','+xmin+ymax+zmax+','+xmin+ymax+zmin+','+xmin+ymin+zmin + ')),'
    y = '((' +xmin+ymax+zmin+','+xmin+ymax+zmax+','+xmax+ymax+zmax+','+xmax+ymax+zmin+','+xmin+ymax+zmin + ')),'
    y_= '((' +xmin+ymin+zmax+','+xmin+ymin+zmin+','+xmax+ymin+zmin+','+xmax+ymin+zmax+','+xmin+ymin+zmax + '))'
    postfix = ")"
    return prefix + z + z_ + x + x_ + y + y_ + postfix    

**Let's insert a simple box (0,0,0),(1,1,1)**

In [8]:
objectWKT = objAsWKT((0,0,0),(1,1,1))

In [9]:
%%sql 
INSERT INTO cube(name,centroid_x,centroid_y,centroid_z,obj_geom) 
VALUES('cubeTest', 0.5, 0.5, 0.5, ST_GeomFromEWKT(:objectWKT));

1 rows affected.


[]

**Validate the box is correctly stored**

In [10]:
%%sql
select ST_AsEWKT(obj_geom) from cube where name = 'cubeTest';

1 rows affected.


st_asewkt
"POLYHEDRALSURFACE(((0 0 1,1 0 1,1 1 1,0 1 1,0 0 1)),((1 0 0,0 0 0,0 1 0,1 1 0,1 0 0)),((1 0 1,1 0 0,1 1 0,1 1 1,1 0 1)),((0 0 0,0 0 1,0 1 1,0 1 0,0 0 0)),((0 1 0,0 1 1,1 1 1,1 1 0,0 1 0)),((0 0 1,0 0 0,1 0 0,1 0 1,0 0 1)))"


**Validate the box is closed**

In [11]:
%%sql 
select ST_IsClosed(obj_geom) from cube where name = 'cubeTest';

1 rows affected.


st_isclosed
True


**Validates it has 6 faces**

In [13]:
%%sql
select ST_numpatches(obj_geom) from cube where name = 'cubeTest';

1 rows affected.


st_numpatches
6


Implementation of the "NearMe" query:
====================================


**Insert sample cubes data**

In [14]:
nb_cubes = 3000000
max_size = 5.0
max_range = 50000.0

Implementation using python/psycopg2 WARNING: insert row by row with auto-commit, so kind of slow (ok for nb_cubes < 100'000):

In [46]:
import psycopg2 as ps
from random import *
conn = ps.connect(host="bbpdbsrv03", database="pspat", user="pspat")
conn.set_isolation_level(0)
cur = conn.cursor()
ins = "insert into cube(name,obj_geom) values(%s,ST_GeomFromEWKT(%s))"
for i in range(nb_cubes):
    x, y, z  = random()*max_range, random()*max_range, random()*max_range
    cube_size = max_size * random()
    name = "cube_at (%d,%d,%d) of size %d" %(x, y, z, cube_size)
    cubewkt = objAsWKT((x,y,z),(x+cube_size,y+cube_size,z+cube_size))
    cur.execute(ins, (name, cubewkt))
cur.close()
conn.close()

Much quicker alternative that bulk insert all rows in one transaction (3M rows loaded in less than 15min):

In [15]:
%%sql
with sim_data as(
	select x xmin, y ymin, z zmin, x+size xmax, y+size ymax, z+size zmax, size from	
	(select id 
		,random()*:max_range as x
		,random()*:max_range as y
		,random()*:max_range as z
		,random()*:max_size as size
	from generate_series(1,:nb_cubes) id) as foo
)
insert into cube(name, centroid_x, centroid_y, centroid_z, obj_geom) 
select 
    'cube at (' || to_char(xmin,'FM999999.00') || ', ' || to_char(ymin,'FM999999.00') || ', ' || to_char(zmin,'FM999999.00') || ')' as name
	,xmin+size, ymin+size, zmin+size
	,ST_GeomFromEWKT( 'SRID=0; POLYHEDRALSURFACE(' || 
		 '((' ||xmin||' '||ymin||' '||zmax||','||xmax||' '||ymin||' '||zmax||','||xmax||' '||ymax||' '||zmax||','||xmin||' '||ymax||' '||zmax||','||xmin||' '||ymin||' '||zmax || ')),' ||
		'((' ||xmax||' '||ymin||' '||zmin||','||xmin||' '||ymin||' '||zmin||','||xmin||' '||ymax||' '||zmin||','||xmax||' '||ymax||' '||zmin||','||xmax||' '||ymin||' '||zmin || ')),' ||
		'((' ||xmax||' '||ymin||' '||zmax||','||xmax||' '||ymin||' '||zmin||','||xmax||' '||ymax||' '||zmin||','||xmax||' '||ymax||' '||zmax||','||xmax||' '||ymin||' '||zmax || ')),' ||
		'((' ||xmin||' '||ymin||' '||zmin||','||xmin||' '||ymin||' '||zmax||','||xmin||' '||ymax||' '||zmax||','||xmin||' '||ymax||' '||zmin||','||xmin||' '||ymin||' '||zmin || ')),' ||
		'((' ||xmin||' '||ymax||' '||zmin||','||xmin||' '||ymax||' '||zmax||','||xmax||' '||ymax||' '||zmax||','||xmax||' '||ymax||' '||zmin||','||xmin||' '||ymax||' '||zmin || ')),' ||
		'((' ||xmin||' '||ymin||' '||zmax||','||xmin||' '||ymin||' '||zmin||','||xmax||' '||ymin||' '||zmin||','||xmax||' '||ymin||' '||zmax||','||xmin||' '||ymin||' '||zmax || ')) )' )		
from sim_data;

3000000 rows affected.


[]

After loading, refresh statistics for Execution plan optimizer

In [16]:
%sql vacuum analyze cube

Done.


[]

**3- Validate query perf**

**Simple implementation with ST_3DDWithin (making use of 'cubeindex' spatial index!)**

In [17]:
%%sql
explain analyze 
select * from cube where ST_3DDWithin(obj_geom, ST_MakePoint(25000, 25000, 25000),1000);

9 rows affected.


QUERY PLAN
Bitmap Heap Scan on cube (cost=136.97..17138.77 rows=289 width=922) (actual time=5.161..106.932 rows=105 loops=1)
Recheck Cond: (obj_geom && '01030000800100000005000000000000000070D740000000000070D740000000000070D740000000000070D740000000000064D940000000000070D740000000000064D940000000000064D940000000000064D940000000000064D940000000000070D740000000000064D940000000000070D740000000000070D740000000000070D740'::geometry)
"Filter: (('010100008000000000006AD84000000000006AD84000000000006AD840'::geometry && st_expand(obj_geom, 1000::double precision)) AND _st_3ddwithin(obj_geom, '010100008000000000006AD84000000000006AD84000000000006AD840'::geometry, 1000::double precision))"
Rows Removed by Filter: 4718
Heap Blocks: exact=4802
-> Bitmap Index Scan on cubeindex (cost=0.00..136.90 rows=4331 width=0) (actual time=3.261..3.261 rows=4823 loops=1)
Index Cond: (obj_geom && '01030000800100000005000000000000000070D740000000000070D740000000000070D740000000000070D740000000000064D940000000000070D740000000000064D940000000000064D940000000000064D940000000000064D940000000000070D740000000000064D940000000000070D740000000000070D740000000000070D740'::geometry)
Planning time: 2.171 ms
Execution time: 107.135 ms


**Optmized implementation with ST_3DDWithin (having the additional 3DBox predicate modifiy the *Filter* step and improves perf!)**

In [18]:
%%sql
explain analyze 
select * 
from cube where ST_3DDWithin(obj_geom, ST_MakePoint(25000, 25000, 25000),1000)
and obj_geom &&& ST_MakeLine(ST_MakePoint(24000, 24000, 24000), ST_MakePoint(26000, 26000, 26000));

9 rows affected.


QUERY PLAN
Bitmap Heap Scan on cube (cost=136.90..17149.53 rows=1 width=922) (actual time=4.174..14.969 rows=105 loops=1)
Recheck Cond: (obj_geom && '01030000800100000005000000000000000070D740000000000070D740000000000070D740000000000070D740000000000064D940000000000070D740000000000064D940000000000064D940000000000064D940000000000064D940000000000070D740000000000064D940000000000070D740000000000070D740000000000070D740'::geometry)
"Filter: ((obj_geom &&& '010200008002000000000000000070D740000000000070D740000000000070D740000000000064D940000000000064D940000000000064D940'::geometry) AND ('010100008000000000006AD84000000000006AD84000000000006AD840'::geometry && st_expand(obj_geom, 1000::double precision)) AND _st_3ddwithin(obj_geom, '010100008000000000006AD84000000000006AD84000000000006AD840'::geometry, 1000::double precision))"
Rows Removed by Filter: 4718
Heap Blocks: exact=4802
-> Bitmap Index Scan on cubeindex (cost=0.00..136.90 rows=4331 width=0) (actual time=2.518..2.518 rows=4823 loops=1)
Index Cond: (obj_geom && '01030000800100000005000000000000000070D740000000000070D740000000000070D740000000000070D740000000000064D940000000000070D740000000000064D940000000000064D940000000000064D940000000000064D940000000000070D740000000000064D940000000000070D740000000000070D740000000000070D740'::geometry)
Planning time: 1.164 ms
Execution time: 15.028 ms


**Validate optimized query along with parameter passing**

In [111]:
#param-ins RefPoint and distance
refX = 10000.0
refY = 10000.0
refZ = 10000.0
distance = 1000.0

#calculate 3D box extreme corners
llx, lly, llz = refX-distance, refY-distance, refZ-distance
urx, ury, urz = refX+distance, refY+distance, refZ+distance

In [112]:
%%sql
select  count(*) as nb_cubes
        ,sum(case when ST_3DDWithin(obj_geom, ST_MakePoint(:refX, :refY, :refZ),:distance) then 1 else 0 end) as nb_within
        ,sum(case when ST_3DDWithin(obj_geom, ST_MakePoint(:refX, :refY, :refZ),:distance) and
        		       (obj_geom &&& ST_MakeLine(ST_MakePoint(:llx, :lly, :llz),ST_MakePoint(:urx, :ury, :urz))) 
        	      then 1  else 0 end) as nb_within_optimized
from cube;

1 rows affected.


nb_cubes,nb_within,nb_within_optimized
3000001,98,98


Strategy for DAO implementation of two query types:
---------------------------------------------------

**1) Find all objects within a certain distance from a given "RefPoint"**


In [113]:
%%sql
select id, name, ST_3DDistance(obj_geom, ST_MakePoint(:refX, :refY, :refZ)) as distance
from cube
where ST_3DDWithin(obj_geom, ST_MakePoint(:refX, :refY, :refZ), :distance)
and obj_geom &&& ST_MakeLine(ST_MakePoint(:llx, :lly, :llz), ST_MakePoint(:urx, :ury, :urz))
order by distance;

98 rows affected.


id,name,distance
1658427,"cube at (9913.65, 9932.07, 10019.07)",105.730173207
326548,"cube at (9839.67, 10173.53, 9994.02)",235.164882207
769791,"cube at (10170.61, 9825.73, 9803.09)",312.3802737
2519024,"cube at (10154.62, 9646.54, 10065.74)",390.164870022
827249,"cube at (9621.09, 9924.41, 9866.67)",403.354307595
1521223,"cube at (9785.25, 10306.28, 9819.62)",411.202632764
1249381,"cube at (10194.80, 10142.91, 9580.67)",479.842791919
1556862,"cube at (10422.79, 10113.21, 10217.34)",488.678124842
2452512,"cube at (10261.17, 9581.43, 9988.69)",489.252222961
984135,"cube at (10249.81, 10212.57, 10366.69)",491.989233481


**2) Find n closest objects from a given "RefPoint"**

In [114]:
limit_nb = 15

In [115]:
%%sql
select * from
(select id,name,ST_3DDistance(obj_geom, ST_MakePoint(:refX, :refY, :refZ)) as distance
from cube
order by distance
limit :limit_nb) as orderset;

15 rows affected.


id,name,distance
1658427,"cube at (9913.65, 9932.07, 10019.07)",105.730173207
326548,"cube at (9839.67, 10173.53, 9994.02)",235.164882207
769791,"cube at (10170.61, 9825.73, 9803.09)",312.3802737
2519024,"cube at (10154.62, 9646.54, 10065.74)",390.164870022
827249,"cube at (9621.09, 9924.41, 9866.67)",403.354307595
1521223,"cube at (9785.25, 10306.28, 9819.62)",411.202632764
1249381,"cube at (10194.80, 10142.91, 9580.67)",479.842791919
1556862,"cube at (10422.79, 10113.21, 10217.34)",488.678124842
2452512,"cube at (10261.17, 9581.43, 9988.69)",489.252222961
984135,"cube at (10249.81, 10212.57, 10366.69)",491.989233481


Unfortunately, the special <-> operator does not work for 3-D (http://boundlessgeo.com/2011/09/indexed-nearest-neighbour-search-in-postgis/).  This slow query does a full table scan + sort and cannot make use of the spatial index.

In [116]:
%%sql
explain analyze 
select id,name,ST_3DDistance(obj_geom, ST_MakePoint(:refX, :refY, :refZ)) as distance
from cube
order by distance
limit :limit_nb;

6 rows affected.


QUERY PLAN
Limit (cost=1228604.93..1228604.97 rows=15 width=898) (actual time=15752.994..15752.999 rows=15 loops=1)
-> Sort (cost=1228604.93..1236104.93 rows=3000002 width=898) (actual time=15752.992..15752.995 rows=15 loops=1)
"Sort Key: (st_3ddistance(obj_geom, '0101000080000000000088C340000000000088C340000000000088C340'::geometry))"
Sort Method: top-N heapsort Memory: 27kB
-> Seq Scan on cube (cost=0.00..1155001.52 rows=3000002 width=898) (actual time=0.034..14972.254 rows=3000001 loops=1)
Total runtime: 15753.032 ms


If needed, we could implement at the DAO layer an iterative approach where we would add both constraints below (both are needed for best perf) and increasing/decreasing distance size as to reach the 'n closest object'

In [118]:
%%sql
explain analyze 
select id,name,ST_3DDistance(obj_geom, ST_MakePoint(:refX, :refY, :refZ)) as distance
from cube
where 
ST_3DDWithin(obj_geom, ST_MakePoint(:refX, :refY, :refZ),:distance) 
and (obj_geom &&& ST_MakeLine(ST_MakePoint(:llx, :lly, :llz), ST_MakePoint(:urx, :ury, :urz)))
order by distance
limit :limit_nb;

11 rows affected.


QUERY PLAN
Limit (cost=16784.91..16784.91 rows=1 width=898) (actual time=14.716..14.718 rows=15 loops=1)
-> Sort (cost=16784.91..16784.91 rows=1 width=898) (actual time=14.714..14.715 rows=15 loops=1)
"Sort Key: (st_3ddistance(obj_geom, '0101000080000000000088C340000000000088C340000000000088C340'::geometry))"
Sort Method: top-N heapsort Memory: 27kB
-> Bitmap Heap Scan on cube (cost=136.17..16784.90 rows=1 width=898) (actual time=3.393..14.624 rows=98 loops=1)
Recheck Cond: (obj_geom && '01030000800100000005000000000000000094C140000000000094C140000000000094C140000000000094C14000000000007CC540000000000094C14000000000007CC54000000000007CC54000000000007CC54000000000007CC540000000000094C14000000000007CC540000000000094C140000000000094C140000000000094C140'::geometry)
"Filter: ((obj_geom &&& '010200008002000000000000000094C140000000000094C140000000000094C14000000000007CC54000000000007CC54000000000007CC540'::geometry) AND ('0101000080000000000088C340000000000088C340000000000088C340'::geometry && st_expand(obj_geom, 1000::double precision)) AND _st_3ddwithin(obj_geom, '0101000080000000000088C340000000000088C340000000000088C340'::geometry, 1000::double precision))"
Rows Removed by Filter: 4715
-> Bitmap Index Scan on cubeindex (cost=0.00..136.17 rows=4234 width=0) (actual time=1.894..1.894 rows=4813 loops=1)
Index Cond: (obj_geom && '01030000800100000005000000000000000094C140000000000094C140000000000094C140000000000094C14000000000007CC540000000000094C14000000000007CC54000000000007CC54000000000007CC54000000000007CC540000000000094C14000000000007CC540000000000094C140000000000094C140000000000094C140'::geometry)


**To find closest objects from another specified object**

The issue here is that ST_Centroid() drops off the Z-coordinate when taking as value the 3D box (either from BOX3d() or ST_3DExtent), so to make this query possible we need to store x,y,z of object centroid explicitly.  

Here's implementation given with the cubes test table:

In [122]:
%%sql
select c.id, c.name
from cube as c,
cube as ref
where ref.id = (select id from cube limit 1)
and c.id != ref.id
and ST_3DDWithin(c.obj_geom, ST_MakePoint(ref.centroid_x, ref.centroid_y, ref.centroid_z), 1000)
and c.obj_geom &&& ST_MakeLine(ST_MakePoint(ref.centroid_x-1000,ref.centroid_y-1000,ref.centroid_z-1000), 
                               ST_MakePoint(ref.centroid_x+1000,ref.centroid_y+1000,ref.centroid_z+1000))
;

9 rows affected.


id,name
217519,"cube at (498.04, 752.12, 2.34)"
653584,"cube at (269.73, 107.48, 241.22)"
1239427,"cube at (94.99, 279.29, 125.82)"
1303678,"cube at (772.63, 236.89, 569.58)"
1528575,"cube at (151.68, 606.24, 573.11)"
1936797,"cube at (69.85, 505.17, 178.09)"
2433245,"cube at (288.92, 264.11, 646.56)"
2648627,"cube at (643.23, 243.64, 242.07)"
2837924,"cube at (191.09, 523.92, 244.39)"


"NearMe" optimized query Execution Plan (same query as above):

In [124]:
%%sql
explain analyze select c.id, c.name from cube as c, cube as ref where ref.id = 9626621 and c.id != ref.id and ST_3DDWithin(c.obj_geom, ST_MakePoint(ref.centroid_x, ref.centroid_y, ref.centroid_z), 1000) and c.obj_geom &&& ST_MakeLine(ST_MakePoint(ref.centroid_x-1000,ref.centroid_y-1000,ref.centroid_z-1000),ST_MakePoint(ref.centroid_x+1000,ref.centroid_y+1000,ref.centroid_z+1000))

10 rows affected.


QUERY PLAN
Nested Loop (cost=10.67..413777.75 rows=1 width=42) (actual time=516.384..516.384 rows=0 loops=1)
-> Seq Scan on cube ref (cost=0.00..412501.03 rows=1 width=28) (actual time=516.383..516.383 rows=0 loops=1)
Filter: (id = 9626621)
Rows Removed by Filter: 3000001
-> Bitmap Heap Scan on cube c (cost=10.67..1276.71 rows=1 width=898) (never executed)
"Recheck Cond: (obj_geom && st_expand(st_makepoint(ref.centroid_x, ref.centroid_y, ref.centroid_z), 1000::double precision))"
"Filter: ((id <> ref.id) AND (st_makepoint(ref.centroid_x, ref.centroid_y, ref.centroid_z) && st_expand(obj_geom, 1000::double precision)) AND (obj_geom &&& st_makeline(st_makepoint((ref.centroid_x - 1000::double precision), (ref.centroid_y - 1000::double precision), (ref.centroid_z - 1000::double precision)), st_makepoint((ref.centroid_x + 1000::double precision), (ref.centroid_y + 1000::double precision), (ref.centroid_z + 1000::double precision)))) AND _st_3ddwithin(obj_geom, st_makepoint(ref.centroid_x, ref.centroid_y, ref.centroid_z), 1000::double precision))"
-> Bitmap Index Scan on cubeindex (cost=0.00..10.67 rows=300 width=0) (never executed)
"Index Cond: (obj_geom && st_expand(st_makepoint(ref.centroid_x, ref.centroid_y, ref.centroid_z), 1000::double precision))"
Total runtime: 516.447 ms
